In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch 
import numpy as np 
import torchvision.transforms as transforms 
import torchvision 
import torch.nn.functional as F
from PIL import Image
import numpy as np
import torch.nn as nn


In [0]:
with open("/content/drive/My Drive/CV/train_labels.csv","r") as files:
  train_labels = [int(i) for i in files.read().split(",")]
  train_labels = np.array(train_labels)-1
with open("/content/drive/My Drive/CV/test_labels.csv","r") as files:
  test_labels = [int(i) for i in files.read().split(",")]
  test_labels = np.array(test_labels)-1

In [0]:
def conv3(in_planes, out_planes, stride=1, groups=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, groups=groups, bias=False)


def conv1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [0]:
class Resunit(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, norm_layer=None):
        super(Resunit, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('Resunit only supports groups=1 and base_width=64')
        self.conv1 = conv3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


In [0]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=8, zero_init_residual=False,
                 groups=1, width_per_group=64, norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.inplanes = 64
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], norm_layer=norm_layer)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, norm_layer=norm_layer)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, norm_layer=norm_layer)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, norm_layer=norm_layer)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, Resunit):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, norm_layer=None):
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet18(pretrained=False, **kwargs):
    model = ResNet(Resunit, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

In [0]:
class Imagedata(torch.utils.data.Dataset):
  def __init__(self,path,labels,transform):
    self.path=path
    self.labels=labels
    self.tr=transform
  def __getitem__(self,index):
    images = Image.open(self.path+str(index+1)+".jpg")
    tr_images = self.tr(images)
    y = self.labels[index]
    p = {"img":tr_images,"label":y}
    return p
  def __len__(self):
    return self.labels.shape[0]
  

In [0]:
def test():
        #To convert data from PIL to tensor
    transform = transforms.Compose(
        [transforms.Resize((224,224)),transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )

    train = Imagedata("/content/drive/My Drive/CV/train/",train_labels,transform = transform)
    trainset = torch.utils.data.DataLoader(train,batch_size=64,shuffle=True,num_workers=2)
    
    test = Imagedata("/content/drive/My Drive/CV/test/",test_labels,transform = transform)
    testset = torch.utils.data.DataLoader(test,batch_size=64,shuffle=True,num_workers=2)
    #Load train and test set:
#     train = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
#     trainset = torch.utils.data.DataLoader(train,batch_size=128,shuffle=True)

#     test = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
#     testset = torch.utils.data.DataLoader(test,batch_size=128,shuffle=False)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    net = resnet18()
    net.to(device)
    costFunc = torch.nn.CrossEntropyLoss()
    optimizer =  torch.optim.Adam(net.parameters(),lr=0.002,weight_decay=1e-3)

    for epoch in range(20):
        closs = 0
        total=0
        correctHits=0
        for i in trainset:
            data = i["img"].to(device)
            output = i["label"].to(device)
            prediction = net(data)
            loss = costFunc(prediction,output)
            closs = loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _,prediction = torch.max(prediction.data,1) 
            total += output.size(0)
            correctHits += (prediction==output).sum().item()
        print("Training  :- for Epoch",epoch+1,"Loss is:",loss.item())
    print('Training Accuracy = '+str((correctHits/total)*100))
    correctHits=0
    total=0
    for i in testset:
        data = i["img"].to(device)
        output = i["label"].to(device)
        prediction = net(data)
        _,prediction = torch.max(prediction.data,1) 
        total += output.size(0)
        correctHits += (prediction==output).sum().item()
    print('Test Accuracy = '+str((correctHits/total)*100))


In [17]:
if __name__ == '__main__':
    test()

cuda:0
Training  :- for Epoch 1 Loss is: 1.288830280303955
Training  :- for Epoch 2 Loss is: 1.0134836435317993
Training  :- for Epoch 3 Loss is: 0.8520622253417969
Training  :- for Epoch 4 Loss is: 0.7764910459518433
Training  :- for Epoch 5 Loss is: 0.7935084104537964
Training  :- for Epoch 6 Loss is: 0.8605446815490723
Training  :- for Epoch 7 Loss is: 0.923723578453064
Training  :- for Epoch 8 Loss is: 0.575831413269043
Training  :- for Epoch 9 Loss is: 0.5524840950965881
Training  :- for Epoch 10 Loss is: 0.3909381926059723
Training  :- for Epoch 11 Loss is: 0.8550170660018921
Training  :- for Epoch 12 Loss is: 0.8101996183395386
Training  :- for Epoch 13 Loss is: 1.0629962682724
Training  :- for Epoch 14 Loss is: 0.3212195038795471
Training  :- for Epoch 15 Loss is: 0.3913416266441345
Training  :- for Epoch 16 Loss is: 0.49861130118370056
Training  :- for Epoch 17 Loss is: 0.3664094805717468
Training  :- for Epoch 18 Loss is: 0.4410684406757355
Training  :- for Epoch 19 Loss is: 